# Recent Images Microservice

This microservice can be used to request multiple tile urls, thumbnail urls and metadata from Google Earth Engine (GEE) in a fast, asynchronus way.

### Overview

On the GFW map page users can currently use the *Sentinel service* to inspect a single satellite image tile for a given area of interest when at high zoom level. The tile url of the image with the lowest ```cloud_score``` within a selected date range is requested from GEE and used on the front end to display the image.

This microservice build on this idea by requesting all images in the date range asynchronously as well as teh corresponding thumbnail urls and associated metadata.

***

### Intended Useage

The microservice is intented to be implemented in 3 stages on the front end:

1) First, do a GET request for the metadata from a GEE image collection (ordered by CLOUDY_PIXEL_PERCENTAGE), as well as the tile url for the best image (i.e. the one with the lowest CLOUDY_PIXEL_PERCENTAGE score)

2) Next POST a payload containing the ```'source'``` values of all images to the ```/v1/recent-tiles/tiles``` endpoint, which returns the tile urls for the associated images.

3) Finally POST the same payload to the ```/v1/recent-tiles/thumbs``` endpoint, which returns all associated thumbnail urls.

***


### Endpoints

The microservice has 3 endpoints:

**/v1/recent-tiles** (GET)
- *Returns tile metadata and a single tile url*
- url params: ```lat, lon, start, end```

**/v1/recent-tiles/tiles** (POST)
- *Returns all tile urls*
- payload: ```list of JSON objects containing 'source' value```

**/v1/recent-tiles/thumbs** (POST)
- *Returns all thumb urls*
- payload: ```list of JSON objects containing 'source' value```





In [1]:
import folium
import os
import ee
import json
import requests
import math
from scipy import misc
import shutil
from pprint import pprint
import requests
from IPython.display import Image


# Request 1

First query requests data for the following params, for example:

```json
    url = "http://localhost:4500/api/v1/recent-tiles"
    params = {'lat':'-16.644','lon':'28.266', 'start':'2016-01-01', 'end': "2016-01-08"}
```

And returns an object of the following format:

```json
"data": [
    {
        "attributes": {
        "boundary_tiles": "https://earthengine.googleapis.com/map/4b1b...",
        "cloud_score": 13.1214,
        "date_time": "2017-01-31 11:52:11Z",
        "instrument": "2017-01-31 11:52:11Z",
        "source": "COPERNICUS/S2/20170131T115211_20170131T115306_T28RCS",
        "thumbnail_url": Null,
        "tile_url": Null
        },
        "id": Null,
        "type": "recent_tiles_url"
    }

```

There may be multiple values in the ```'data'``` list (expect one image per 8 days approx.)


In [193]:
%%time
# Request metadata

url = f"http://localhost:4500/api/v1/recent-tiles"
params= {'lat':'-16.644','lon':'28.266', 'start':'2016-01-01', 'end': "2016-01-08"}
r = requests.get(url, params=params)
r.status_code
pprint.pprint(r.json())

data = r.json().get('data')

{'data': [{'attributes': {'boundary_url': 'https://earthengine.googleapis.com/map/4b1b9c6f82d50796562521502bc4d9a2/{z}/{x}/{y}?token=95527549cde3e5cf1f677852d3931e67',
                          'cloud_score': 41.0,
                          'date_time': '2016-01-07 11:54:14Z',
                          'instrument': 'Sentinel-2A',
                          'source': 'COPERNICUS/S2/20160107T115414_20160108T134003_T28RCS',
                          'thumbnail_url': None,
                          'tile_url': 'https://earthengine.googleapis.com/map/9eafd94e2481677f962db817b34b0a60/{z}/{x}/{y}?token=bce7202e887afb675e7e336194aa6269'},
           'id': None,
           'type': 'recent_tiles_data'}]}
CPU times: user 5.78 ms, sys: 3.06 ms, total: 8.84 ms
Wall time: 2.66 s


# Creating Payload for POST

On the front end, the response must be used to buld a list of objects containing the ```'source'``` values of the images in the collection.

*e.g.*

```json
{'source_data':
    
    [
        {"source": "COPERNICUS/S2/20170131T11..."},
        {"source": "COPERNICUS/S2/20170131T12..."},
        {"source": "COPERNICUS/S2/20170131T13..."},    
        
        ...
      
        {"source": "COPERNICUS/S2/20170131T1N..."}
    ]
 }

```

### IMPORTANT

- *The json must have the key ```'source_data'```!*

- Must have a the following header: ```headers={'Content-Type': 'application/json'}```

In [194]:
#example of how to construct the source list

source_list = []

for d in data:
    source_list.append({'source': d.get('attributes').get('source')}) 
source_list[:]

[{'source': 'COPERNICUS/S2/20160107T115414_20160108T134003_T28RCS'}]

In [195]:
%%time
# Request tiles using the payload

url = f"http://localhost:4500/api/v1/recent-tiles/tiles"
payload = {'source_data':source_list}
r = requests.post(url, data=json.dumps(payload), headers={'Content-Type': 'application/json'})
r.status_code
pprint.pprint(r.json())

tile_data = r.json().get('data')

{'data': {'attributes': [{'source_id': 'COPERNICUS/S2/20160107T115414_20160108T134003_T28RCS',
                          'tile_url': 'https://earthengine.googleapis.com/map/9eafd94e2481677f962db817b34b0a60/{z}/{x}/{y}?token=7f76b0102e0e9bcf82f197faa2fdb089'}],
          'id': None,
          'type': 'recent_tiles_url'}}
CPU times: user 5.73 ms, sys: 2.23 ms, total: 7.96 ms
Wall time: 947 ms


### Tiles Response

At this stage the data now looks like:

```json
{'data': {
    'attributes': [{"source": "COPERNICUS/S2/20170131T11...",
                    "tile_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   },{
                    "source": "COPERNICUS/S2/20170131T12...",
                    "tiles_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   },           
                      ...
      
                    {
                    "source": "COPERNICUS/S2/20170131T1N...",
                    "tiles_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   }],
          'id': None,
          'type': 'recent_tiles_url'}}
          }
}
```

In [196]:
%%time

# Request thumbs using the payload

url = f"http://localhost:4500/api/v1/recent-tiles/thumbs"
payload = {'source_data':source_list}
r = requests.post(url, data=json.dumps(payload), headers={'Content-Type': 'application/json'})
r.status_code
pprint.pprint(r.json())

thumb_data = r.json().get('data')

{'data': {'attributes': [{'source': 'COPERNICUS/S2/20160107T115414_20160108T134003_T28RCS',
                          'thumbnail_url': 'https://earthengine.googleapis.com/api/thumb?thumbid=f02d0f1a21834fd26c86a4d3f9bc4c05&token=5d5426aeb181c5f91e9b749105bb7487'}],
          'id': None,
          'type': 'recent_thumbs_url'}}
CPU times: user 5.31 ms, sys: 2.01 ms, total: 7.32 ms
Wall time: 1.85 s


### Thumbs Response

At this stage the data now looks like:

```json
{'data': {
    'attributes': [{"source": "COPERNICUS/S2/20170131T11...",
                    "thumbnail_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   },{
                    "source": "COPERNICUS/S2/20170131T12...",
                    "thumbnail_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   },           
                      ...
      
                    {
                    "source": "COPERNICUS/S2/20170131T1N...",
                    "thumbnail_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   }],
          'id': None,
          'type': 'recent_thumbs_url'}}
          }
}
```

# Image Example for Map

In [206]:
# Returned First Image and Boundary

dt = data[0].get('attributes').get('date_time')
boundary = data[0].get('attributes').get('boundary_url')
sentinel_image = data[0].get('attributes').get('tile_url')

In [202]:
sentinel_map = folium.Map(location=[float(params['lon']), float(params['lat'])], zoom_start=9, tiles='Mapbox Bright' )
sentinel_map.add_tile_layer(tiles=sentinel_image, max_zoom=19, min_zoom=6, attr="Live EE tiles")
sentinel_map.add_tile_layer(tiles=boundary, max_zoom=19, min_zoom=6, attr="Live EE tiles")
sentinel_map

In [214]:
# Corresponding Thumbnail

thumbnail_image = thumb_data.get('attributes')[0].get('thumbnail_url')
Image(url=thumbnail_image)